<center> <font color=red > <font size = 7> NFTs on AlgoRand  </font> <br> <br> <font color= b> <font size = 5> Algorand Standar Asset (ASA) </font>


<img style="float: right;" src="files/blocks.jpeg">

>El protocolo Algorand permite la creación de activos en la cadena que se benefician de la misma seguridad, compatibilidad, velocidad y facilidad de uso que el Algo. El nombre oficial de los activos en Algorand es Algorand Standard Assets (*ASA*).

>Con los activos estándar de Algorand puedes representar stablecoins, loyalty points, system credits, and in-game points, por nombrar algunos ejemplos. También puedes representar activos únicos, como la escritura de una casa, objetos coleccionables, piezas únicas en una cadena de suministro, etc. También hay una funcionalidad opcional para colocar restricciones de transferencia en un activo que ayuda a apoyar los casos de uso de valores, cumplimiento y certificación.

>Los activos que representan muchos del mismo tipo, como una stablecoin, pueden denominarse activos fungibles. Los activos únicos se denominan activos no fungibles (NFT).

>La NFT es una representación única y no intercambiable de un activo físico y digital en la cadena de bloques. Las NFT tienen un amplio alcance y el concepto puede aplicarse a los derechos de autor de las obras de arte, los bienes inmuebles, las representaciones de la identidad digital y los objetos de colección.

>El método principal que un desarrollador o usuario de Algorand puede utilizar para construir y desplegar una NFT es mediante la función ASA. Esta función es una primitiva de capa 1 que permite crear una NFT o FT en segundos. Sólo se necesita una transacción en la blockchain de Algorand para crear un NFT. Esta transacción describe las propiedades, tanto mutables como inmutables, del token.



## Immutable Asset Parameters

-Creator (required)

-AssetName (optional, but recommended)

-UnitName (optional, but recommended)

-Total (required)

-Decimals (required)

-DefaultFrozen (required)

-URL (optional)

-MetaDataHash (optional)


## Mutable Asset Parameters

-Manager Address

-Reserve Address

-Freeze Address

-Clawback Address


## Sistema de transacción

>Crear transacción ASA (Algorand Standard Asset):

Cree activos usando los SDKs (Software Development Kit) o el objetivo.

>Modificación de un activo:

Sólo se pueden modificar las cuentas del manager, reserve, freeze and clawback

>Recibir un activo:

Antes de que una cuenta pueda recibir un activo específico, debe optar por recibirlo.

>Transferencia de un activo:

Los activos pueden transferirse entre cuentas que hayan optado por recibirlos.

<hr style="border:0.02in solid red"> </hr>

In [2]:
import pandas as pd
import json
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn

<font color= "brown"> <font size = 5> CREATE ACCOUNTS </font>

Se crean cuentas genéricas para el desarrollo del código. Organizador y Equipo es necesario que tengan un Wallet para las transacciones en blockchain.

In [1]:
# frase mnemotécnica para Organizador y Equipo
mnemonic1 = "canal enact luggage spring similar zoo couple stomach shoe laptop middle wonder eager monitor weather number heavy skirt siren purity spell maze warfare ability ten"
mnemonic2 = "beauty nurse season autumn curve slice cry strategy frozen spy panic hobby strong goose employ review love fee pride enlist friend enroll clip ability runway"
mnemonic3 = "picnic bright know ticket purity pluck stumble destroy ugly tuna luggage quote frame loan wealth edge carpet drift cinnamon resemble shrimp grain dynamic absorb edge"

In [3]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [mnemonic1, mnemonic2, mnemonic3]:
    accounts[counter] = {}
    accounts[counter]['pk'] = mnemonic.to_public_key(m)
    accounts[counter]['sk'] = mnemonic.to_private_key(m)
    counter += 1

In [4]:
# Specify your node address and token. This must be updated.
algod_address = "http://hackathon.algodev.network:9100"
algod_token = "ef920e2e7e002953f4b29a8af720efe8e4ecc75ff102b165e0472834b25832c1"

In [5]:
# Initialize an algod client
algod_client = algod.AlgodClient(algod_token=algod_token, algod_address=algod_address)

def wait_for_confirmation(client, txid):
    """
    Utility function to wait until the transaction is
    confirmed before proceeding.
    """
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(txid, txinfo.get('confirmed-round')))
    return txinfo

#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):    
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1       
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

print("Organizador address: {}".format(accounts[1]['pk']))
print("Organizador address: {}".format(accounts[2]['pk']))
print("Equipo address: {}".format(accounts[3]['pk']))


Organizador address: ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ
Organizador address: AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4
Equipo address: IWR4CLLCN2TIVX2QPVVKVR5ER5OZGMWAV5QB2UIPYMPKBPLJZX4C37C4AA


<font color= "brown"> <font size = 5> CREATE ASSET </font>


Cree activos usando los SDKs (Software Development Kit) o el objetivo. Cuando se utilizan los SDKs se suministran todos los parámetros de creación. Con goal, la gestión de las distintas direcciones asociadas al activo debe hacerse después de ejecutar la creación de un activo.

In [6]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# Account 1 creates an asset called nft and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    total=12,
    default_frozen=False,
    unit_name="CRIPTOFUTBOL",
    asset_name="CFUT",
    manager=accounts[2]['pk'],
    reserve=accounts[2]['pk'],
    freeze=accounts[2]['pk'],
    clawback=accounts[2]['pk'],
    url="https://path/to/my/asset/details", 
    decimals=0)
# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])

# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client,txid)

try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

AGRGCB55H2JK4WF36SUVZ4NL7NMUYREASHMULLFHZIZ45RN75I2Q
Waiting for confirmation
Transaction AGRGCB55H2JK4WF36SUVZ4NL7NMUYREASHMULLFHZIZ45RN75I2Q confirmed in round 15337840.
Asset ID: 19144026
{
    "clawback": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "manager": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "name": "nft",
    "reserve": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "total": 12,
    "unit-name": "NFT",
    "url": "https://path/to/my/asset/details"
}
Asset ID: 19144026
{
    "amount": 12,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": false
}


![general](files/create.png)

<font color= "brown"> <font size = 5> CHANGE MANAGER </font>

Una vez creado un activo, sólo pueden modificarse las cuentas de gestor, reserva, congelación y recuperación. Todos los demás parámetros están bloqueados durante la vida del activo. Si alguna de estas direcciones se establece como "", esa dirección se borrará y no podrá restablecerse nunca durante la vida del activo. Sólo la cuenta del gestor puede realizar cambios de configuración y debe autorizar la transacción.

In [7]:
# The current manager(Account 2) issues an asset configuration transaction that assigns Account 1 as the new manager.
# Keep reserve, freeze, and clawback address same as before, i.e. account 2
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# asset_id = 328952;

txn = AssetConfigTxn(
    sender=accounts[2]['pk'],
    sp=params,
    index=asset_id, 
    manager=accounts[1]['pk'],
    reserve=accounts[2]['pk'],
    freeze=accounts[2]['pk'],
    clawback=accounts[2]['pk'])
# sign by the current manager - Account 2
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)

# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

# Check asset info to view change in management. manager should now be account 1
print_created_asset(algod_client, accounts[1]['pk'], asset_id)

6YS2A2LGYRDJR33E7ZKCSXNFNLWMEM7Y57CO6LAQWWWAGVRKNM5A
Waiting for confirmation
Transaction 6YS2A2LGYRDJR33E7ZKCSXNFNLWMEM7Y57CO6LAQWWWAGVRKNM5A confirmed in round 15337844.
Asset ID: 19144026
{
    "clawback": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "manager": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "name": "nft",
    "reserve": "AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4KC6XL5PWW5K4",
    "total": 12,
    "unit-name": "NFT",
    "url": "https://path/to/my/asset/details"
}


<font color= "brown"> <font size = 5> OPT-IN </font>

Antes de que una cuenta pueda recibir un activo específico, debe optar por recibirlo. Una transacción opt-in coloca una tenencia de activos de 0 en la cuenta y aumenta su saldo mínimo en 100.000 microAlgos. Una transacción opt-in es simplemente una transferencia de activos con una cantidad de 0, tanto hacia como desde la cuenta opt-in.

In [8]:
# Check if asset_id is in account 3's asset holdings prior
# to opt-in
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

account_info = algod_client.account_info(accounts[3]['pk'])
holding = None
idx = 0
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    idx = idx + 1    
    if (scrutinized_asset['asset-id'] == asset_id):
        holding = True
        break

if not holding:

    # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=accounts[3]['pk'],
        sp=params,
        receiver=accounts[3]["pk"],
        amt=0,
        index=asset_id)
    stxn = txn.sign(accounts[3]['sk'])
    txid = algod_client.send_transaction(stxn)
    print(txid)
    # Wait for the transaction to be confirmed
    wait_for_confirmation(algod_client, txid)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

XAJPGRUIOS3VKQ2YHUVDYZSQJKHA5KPNPNP3NKVWFQ3GWYQ5YWTQ
Waiting for confirmation
Transaction XAJPGRUIOS3VKQ2YHUVDYZSQJKHA5KPNPNP3NKVWFQ3GWYQ5YWTQ confirmed in round 15337847.
Asset ID: 19144026
{
    "amount": 0,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": false
}


<font color= "brown"> <font size = 5> TRANSFER ASSET </font>

Los activos pueden ser transferidos entre cuentas que han optado por recibir el activo. Son análogas a las transacciones de pago estándar, pero para los activos estándar de Algorand.

In [9]:
# transfer asset of 10 from account 1 to account 3
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True
txn = AssetTransferTxn(
    sender=accounts[1]['pk'],
    sp=params,
    receiver=accounts[3]["pk"],
    amt=10,
    index=asset_id)
stxn = txn.sign(accounts[1]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)
# The balance should now be 10.
print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

NK7M7IALHG4H747EAMDEVMRSTSCSIIIDD3AWX4SWSESZPHQY47ZA
Waiting for confirmation
Transaction NK7M7IALHG4H747EAMDEVMRSTSCSIIIDD3AWX4SWSESZPHQY47ZA confirmed in round 15337849.
Asset ID: 19144026
{
    "amount": 10,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": false
}


<font color= "brown"> <font size = 5> FREEZE ASSET </font>

La congelación o descongelación de un activo para una cuenta requiere una transacción firmada por la cuenta congelada. 

In [10]:
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True
# The freeze address (Account 2) freezes Account 3's latinum holdings.

txn = AssetFreezeTxn(
    sender=accounts[2]['pk'],
    sp=params,
    index=asset_id,
    target=accounts[3]["pk"],
    new_freeze_state=True   
    )
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)
# The balance should now be 10 with frozen set to true.
print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

AU2VE4NX7UG74LQ26A4MHUQP2WVSXIFBPRRWBK7VTZ4O7NTFHXLA
Waiting for confirmation
Transaction AU2VE4NX7UG74LQ26A4MHUQP2WVSXIFBPRRWBK7VTZ4O7NTFHXLA confirmed in round 15337851.
Asset ID: 19144026
{
    "amount": 10,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": true
}


<font color= "brown"> <font size = 5> REVOKE ASSET </font>

La revocación de un activo para una cuenta elimina un número específico del activo de la cuenta de destino de la revocación. La revocación de un activo de una cuenta requiere especificar un remitente de activos (la cuenta de destino de la revocación) y un receptor de activos (la cuenta a la que transferir los fondos de nuevo).

In [11]:
# The clawback address (Account 2) revokes 10 latinum from Account 3 and places it back with Account 1.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# Must be signed by the account that is the Asset's clawback address
txn = AssetTransferTxn(
    sender=accounts[2]['pk'],
    sp=params,
    receiver=accounts[1]["pk"],
    amt=10,
    index=asset_id,
    revocation_target=accounts[3]['pk']
    )
stxn = txn.sign(accounts[2]['sk'])
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)
# The balance of account 3 should now be 0.
# account_info = algod_client.account_info(accounts[3]['pk'])
print("Equipo")
print_asset_holding(algod_client, accounts[3]['pk'], asset_id)

# The balance of account 1 should increase by 10 to 1000.
print("Organizador")
print_asset_holding(algod_client, accounts[1]['pk'], asset_id)

PZV4BC5WHQFEBQMZDYJVJ62U2U6C4QS6YSXQEZIU47PK4ESTO3MQ
Waiting for confirmation
Transaction PZV4BC5WHQFEBQMZDYJVJ62U2U6C4QS6YSXQEZIU47PK4ESTO3MQ confirmed in round 15337854.
Equipo
Asset ID: 19144026
{
    "amount": 0,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": true
}
Organizador
Asset ID: 19144026
{
    "amount": 12,
    "asset-id": 19144026,
    "creator": "ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55EVKZWOWSM7ABQ",
    "is-frozen": false
}


<font color= "brown"> <font size = 5> DESTROY ASSET </font>

Los activos creados sólo pueden ser destruidos por la cuenta del gestor de activos. Todos los activos deben ser propiedad del creador del activo antes de que éste pueda ser eliminado.

In [12]:
# With all assets back in the creator's account,
# the manager (Account 1) destroys the asset.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# Asset destroy transaction
txn = AssetConfigTxn(
    sender=accounts[1]['pk'],
    sp=params,
    index=asset_id,
    strict_empty_address_check=False
    )

# Sign with secret key of creator
stxn = txn.sign(accounts[1]['sk'])
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)
# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

# Asset was deleted.
try:
    print("El equipo debe hacer una transacción por un importe de 0, " )
    print("con un close_assets_to a la cuenta del creador, para borrarla de su contabilidad")
    print("En el caso de Organizador, no debería imprimirse nada después de esto, ya que el activo se destruye en la cuenta del creador")
   
    print_asset_holding(algod_client, accounts[1]['pk'], asset_id)
    print_created_asset(algod_client, accounts[1]['pk'], asset_id)
    # asset_info = algod_client.asset_info(asset_id)
except Exception as e:
    print(e)

6NW7HYNTSISFOLP2NONQCSKTB4AY7N5RFESOQBHXQ52P3MKA2CBQ
Waiting for confirmation
Transaction 6NW7HYNTSISFOLP2NONQCSKTB4AY7N5RFESOQBHXQ52P3MKA2CBQ confirmed in round 15337857.
El equipo debe hacer una transacción por un importe de 0, 
con un close_assets_to a la cuenta del creador, para borrarla de su contabilidad
En el caso de Organizador, no debería imprimirse nada después de esto, ya que el activo se destruye en la cuenta del creador


In [13]:
general = {'Circulating Supply':[0],
        'Total Supply':[12],
        'Unit Name':['NFT'],
        'Token ID':[asset_id],
        'Reserve Address':[accounts[3]['pk']],
        'Creator Address':[accounts[1]['pk']],
        'Default Frozen':['is-false'],
        'Number of Transfers':[0]}
pd.DataFrame(general)

,Circulating Supply,Total Supply,Unit Name,Token ID,Reserve Address,Creator Address,Default Frozen,Number of Transfers
0,0,12,NFT,19144026,IWR4CLLCN2TIVX2QPVVKVR5ER5OZGMWAV5QB2UIPYMPKBP...,ATTR6RUEHHBHXKUHT4GUOYWNBVDV2GJ5FHUWCSFZLHD55E...,is-false,0


In [14]:
technical_info = {'Manager Address':[accounts[2]['pk']],
        'Freeze Address':[accounts[2]['pk']],
        'Creation Tx':[txid],
        'Clawback Account':[accounts[2]['pk']],
        'RDecimals':[0]}
        
pd.DataFrame(technical_info)

,Manager Address,Freeze Address,Creation Tx,Clawback Account,RDecimals
0,AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4...,AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4...,6NW7HYNTSISFOLP2NONQCSKTB4AY7N5RFESOQBHXQ52P3M...,AK6Q33PDO4RJZQPHEMODC6PUE5AR2UD4FBU6TNEJOU4UR4...,0


![general](files/asa.png)



![general](files/nft1.png)



Una vez que el bloque que contiene la transacción génesis del ASA se ha añadido a la cadena de bloques, el activo cobra vida devolviendo el ID de la transacción génesis y su ID de activo. Ahora puedes encontrar tu ASA entre los activos estándar de Algorand que aparecen en el explorador de bloques. https://algoexplorer.io/
El ID del activo es la identidad única de nuestro ASA que acabamos de crear. En bases de datos, la identidad única del registro en una base de datos está garantizada por una clave primaria que es inmutable y diferente de todos los demás registros. El identificador de activos funciona de la misma manera. Otra persona puede crear ASAs exactamente con el mismo nombre de activo
nombre de activo y nombre de unidad de activo que ya existen en la cadena de bloques. Sin embargo, lo que realmente diferencia su ASA de los demás es el ID de activo.



<hr style="border:0.02in solid red"> </hr>

## Links

https://developer.algorand.org/articles/building-nfts-on-algorand/


https://wallet.myalgo.com/


https://algodesk.io/#/portal/home


https://algoexplorer.io/asset/273448084